# Lab 4: Develop a Finance Analyst Multi-Agent System

In this lab, we’ll build a multi-agent system where a team of agents works together to generate detailed analyst reports about corporate finance data. The system consists of three task agents plus an orchestrator:

1. FinanceDataAgent – This agent searches an Azure AI Search index to retrieve recent financial information and performance data for your company.
2. AnalystReportAgent – This agent writes a detailed analyst report synthesizing the retrieved data, including insights on financial performance, Finance trends, and risk analysis.
3. ValidationAgent – This agent validates that the final report includes a detailed risk assessment.
4. FinanceOrchestratorAgent – The orchestrator that communicates with the above agents to create the final analyst report.

We use the Azure AI Agent Service for the individual task agents and Semantic Kernel to build the orchestrator.

### Part 1: Create the FinanceData, AnalystReport, and Validation Agents

In [8]:
import os
import logging
import json
from semantic_kernel.functions import kernel_function
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import AzureAISearchTool
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

load_dotenv()

True

#### Step 2: Create the FinanceDataAgent

This agent from Lab 2a searches your existing vector store retrieves relevant financial information.

In [9]:
class FinanceDataAgent:
    """
    A class to represent the Finance Data Agent.
    """
    @kernel_function(description='An agent that searches for financial information from internal data.')
    def search_finance_data(self, data_description: str) -> str:
        """
        Searches an Azure AI Search index for financial data about a company.
       
        Parameters:
        data_description (str): The name of the company to search for.
       
        Returns:
        last_msg (json): The final message containing relevant financial information.
        """
        print("Calling FinanceDataAgent...")
       
        project_client = AIProjectClient.from_connection_string(
            credential=DefaultAzureCredential(),
            conn_str=os.environ["AIPROJECT_CONNECTION_STRING"],
        )
    
        #retrieve agent by name
        agent_name = "Sales Analyst Agent"

        # Check if the agent already exists
        agents = project_client.agents.list_agents()
        fin_agent = next((a for a in agents.data if a.name == agent_name), None)

        if fin_agent is None:
            return f"Agent '{agent_name}' not found."
        
        thread = project_client.agents.create_thread()
       
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=f"P{data_description}.",
        )
       
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=fin_agent.id)
       
        if run.status == "failed":
            print(f"Run failed: {run.last_error}")
       
        #project_client.agents.delete_agent(fin_agent.id)
       
        messages = project_client.agents.list_messages(thread_id=thread.id)
        last_msg = messages.get_last_text_message_by_role("assistant")
       
        print("FinanceDataAgent completed successfully.")
        return last_msg

#### Step 3: Create the AnalystReportAgent

In [10]:
class AnalystReportAgent:
    """
    A class to represent the Analyst Report Agent.
    """
    @kernel_function(description='An agent that writes detailed analyst reports on finance data.')
    def write_report(self, finance_data:str, data_description: str) -> str:
        """
        Writes a detailed analyst report for a company.
        
        Parameters:
        finance_data (str): the financial data to be included in the report.
        data_description (str): the topic of the report.

        Returns:
        last_msg (json): The final message containing the detailed analyst report.
        """
        print("Calling AnalystReportAgent...")
        
        project_client = AIProjectClient.from_connection_string(
            credential=DefaultAzureCredential(),
            conn_str=os.environ["AIPROJECT_CONNECTION_STRING"],
        )
        
        report_agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="analyst-report-agent",
            instructions="You are a helpful agent specializing in writing comprehensive analyst reports. Your report should include analysis key financial performance, and a thorough risk assessment.",
        )
        
        thread = project_client.agents.create_thread()
        
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=f"Write a detailed analyst report regarding {data_description}. Include insights into Finance trends, key financial metrics, and a risk assessment retrieved from the {finance_data}.",
        )
        
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=report_agent.id)
        
        if run.status == "failed":
            print(f"Run failed: {run.last_error}")
        
        project_client.agents.delete_agent(report_agent.id)
        
        messages = project_client.agents.list_messages(thread_id=thread.id)
        last_msg = messages.get_last_text_message_by_role("assistant")
        
        print("AnalystReportAgent completed successfully.")
        return last_msg

#### Step 4: Create the Validation Agent

This agent validates that the generated analyst report meets our standards – specifically, it checks that the report includes a detailed risk assessment.

In [11]:
class ValidationAgent:
    """
    A class to represent the Validation Agent.
    """
    @kernel_function(description='An agent that runs validation checks to ensure that the generated analyst report meets required standards.')
    def validate_report(self, report: str) -> str:
        """
        Validates the generated analyst report.
        Requirement: The report must include a detailed risk assessment.
        
        Parameters:
        report (str): The analyst report produced by the AnalystReportAgent.
        
        Returns:
        last_msg (json): The final message containing the validation result.
        """
        print("Calling ValidationAgent...")
        
        project_client = AIProjectClient.from_connection_string(
            credential=DefaultAzureCredential(),
            conn_str=os.environ["AIPROJECT_CONNECTION_STRING"],
        )
        
        validation_agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="validation-agent",
            instructions="You are an expert agent that validates analyst reports. Return 'Pass' if the report includes a detailed risk assessment and meets all reporting standards, otherwise return 'Fail'. You must only return 'Pass' or 'Fail'.",
        )
        
        thread = project_client.agents.create_thread()
        
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=f"Validate that the generated analyst report includes a detailed risk assessment. Here is the report: {report}",
        )
        
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=validation_agent.id)
        
        if run.status == "failed":
            print(f"Run failed: {run.last_error}")
        
        project_client.agents.delete_agent(validation_agent.id)
        
        messages = project_client.agents.list_messages(thread_id=thread.id)
        last_msg = messages.get_last_text_message_by_role("assistant")
        
        print("ValidationAgent completed successfully.")
        return last_msg

### Part 2: Create a Multi-Agent System for Generating Analyst Reports

The orchestrator (FinanceOrchestratorAgent) will coordinate the above agents to generate an analyst report. When you run the notebook, you will be prompted for a publicly traded company name. If the generated report meets the validation criteria, it will be saved as a file.

Try the following prompts:

##

In [12]:
# Environment variables to connect to the gpt-4o model
deployment_name = os.environ["CHAT_MODEL"]
endpoint = os.environ["CHAT_MODEL_ENDPOINT"]
api_key = os.environ["CHAT_MODEL_API_KEY"]

async def main():
    # Initialize the Semantic Kernel.
    kernel = Kernel()
    
    # Add services and plugins to the kernel.
    service_id = "orchestrator_agent"
    kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name=deployment_name, endpoint=endpoint, api_key=api_key))
    kernel.add_plugin(AnalystReportAgent(), plugin_name="AnalystReportAgent")
    kernel.add_plugin(FinanceDataAgent(), plugin_name="FinanceDataAgent")
    kernel.add_plugin(ValidationAgent(), plugin_name="ValidationAgent")
    
    settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
    
    # Create the FinanceOrchestratorAgent to coordinate the agents.
    agent = ChatCompletionAgent(
        service_id="orchestrator_agent",
        kernel=kernel,
        name="FinanceOrchestratorAgent",
        instructions=f"""
        You are an agent designed to create detailed analyst reports for finance data. The user will provide a data_description, and you will generate an analyst report by orchestrating the plugin agents:
        
        - AnalystReportAgent: Formats finance data into report, writes comprehensive analyst reports.
        - FinanceDataAgent: Retrieves financial data and Finance performance information.
        - ValidationAgent: Checks that the report includes a detailed risk assessment.
        
        It is crucial that the final report includes a risk assessment section. If the report does not meet this requirement, it should be rejected.
        Format your final response as a JSON object with two attributes, report_was_generated and content:
        
        - report_was_generated: Boolean that is true if a valid report was produced, otherwise false.
        - content: A string containing the source finance data from the FinanceDataAgent and detailed analyst report if valid, or an error message if not.
        
        Example response:
        {{"report_was_generated": false, "content": "The analyst report for the requested company could not be generated because it lacks a risk assessment section."}}
        
        Your response must be a single valid JSON object using lowercase booleans (true/false) and double quotes for all keys and string values.
        """,
        execution_settings=settings,
    )
    
    history = ChatHistory()
    
    is_complete = False
    while not is_complete:
        print("FinanceOrchestratorAgent is starting...")
        
        user_input = input("Hello. Please provide the description to generate an analyst report. Type 'exit' to end the conversation: ")
        if not user_input:
            continue
        
        if user_input.lower() == "exit":
            is_complete = True
            break
        
        history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
        
        async for response in agent.invoke(history=history):
            fixed_content = response.content.replace("False", "false").replace("True", "true")
            print(f"Response: {fixed_content}")
            response_json = json.loads(fixed_content)
            report_was_generated = response_json['report_was_generated']
            report_content = response_json['content']
            
            if report_was_generated:
                report_name = f"Analyst Report - {user_input}.md"

                os.makedirs("output", exist_ok=True)
                with open("output/{}".format(report_name), "w") as f:
                    f.write(report_content)
                print(f"The analyst report for {user_input} has been generated. Please check the file {report_name}.")
            else:
                print(report_content)
                
await main()

FinanceOrchestratorAgent is starting...
Calling FinanceDataAgent...
FinanceDataAgent completed successfully.
Calling AnalystReportAgent...
AnalystReportAgent completed successfully.
Calling ValidationAgent...
ValidationAgent completed successfully.
Response: {"report_was_generated": true, "content": "Financial data: Here are summaries of the product sales data for Canada:\\n\\n1. **Paseo (Low Discount Band):**\\n   - Small Business: Sold 1,916 units and achieved gross sales of $574,800, with a profit of $84,304.\\n   - Government: Sold 2,852 units with gross sales amounting to $998,200, resulting in a profit of $236,716.\\n   - Enterprise: Sold 2,729 units, generating gross sales of $341,125 and a profit of $6,822.5.\\n\\n2. **Paseo (High Discount Band):**\\n   - Government: Sold 2,104.5 units, gross sales reached $736,575 with a profit of $108,382.\\n   - Government: Sold 2,394 units and had gross sales of $47,880 with a profit of $18,673.2.\\n\\n3. **Velo (Medium Discount Band):**\\n